In [1]:
import ollama
from fastapi import FastAPI
from pydantic import BaseModel
from fastapi.middleware.cors import CORSMiddleware
from typing import List, Optional

app = FastAPI()

# CORS
app.add_middleware(
    CORSMiddleware,
    allow_origins=["*"],
    allow_methods=["*"],
    allow_headers=["*"],
)

# Define structure of a single message
class Message(BaseModel):
    sender_name: str
    sender_email: str
    message: str
    timestamp: Optional[str] = None  # ISO string, optional

# Main request body
class EmailRequest(BaseModel):
    conversation_thread: List[Message]  # Full conversation thread
    reply_style: str  # e.g., "formal", "friendly", etc.
    prompt: Optional[str] = None       # Optional custom prompt to override
    new_message: Optional[str] = None  # Optional new message at the end

@app.post("/generate-email")
def generate_email(req: EmailRequest):
    # Build the conversation thread text
    thread_text = ""


In [2]:
{
  "conversation_thread": [
    {
      "sender_name": "Alice",
      "sender_email": "alice@example.com",
      "message": "Hey Bob, can you share the report by Monday?"
    },
    {
      "sender_name": "Bob",
      "sender_email": "bob@example.com",
      "message": "Sure, I'll send it by end of day."
    },
    {
      "sender_name": "Alice",
      "sender_email": "alice@example.com",
      "message": "Thanks! Please include the latest sales figures too."
    }
  ],
  "reply_style": "formal",
  "new_message": "Here’s the final version of the report. Let me know if anything else is needed."
}


{'conversation_thread': [{'sender_name': 'Alice',
   'sender_email': 'alice@example.com',
   'message': 'Hey Bob, can you share the report by Monday?'},
  {'sender_name': 'Bob',
   'sender_email': 'bob@example.com',
   'message': "Sure, I'll send it by end of day."},
  {'sender_name': 'Alice',
   'sender_email': 'alice@example.com',
   'message': 'Thanks! Please include the latest sales figures too.'}],
 'reply_style': 'formal',
 'new_message': 'Here’s the final version of the report. Let me know if anything else is needed.'}

In [3]:
app = FastAPI()

# Enable CORS
app.add_middleware(
    CORSMiddleware,
    allow_origins=["*"],
    allow_methods=["*"],
    allow_headers=["*"],
)

# Message structure
class Message(BaseModel):
    sender_name: str
    sender_email: str
    message: str
    timestamp: Optional[str] = None

# Main request model
class EmailRequest(BaseModel):
    conversation_thread: List[Message]
    reply_style: str  # e.g., "formal", "friendly", etc.

@app.post("/generate-email")
def generate_email(req: EmailRequest):
    # Separate latest message from previous history
    *history_messages, latest_message = req.conversation_thread

    # Build history text
    history_text = ""
    for msg in history_messages:
        history_text += f"{msg.sender_name} ({msg.sender_email}): {msg.message}\n"

    # Add the message that needs to be replied to
    last_sender = latest_message.sender_name
    last_email = latest_message.sender_email
    last_msg = latest_message.message

    # Construct prompt
    prompt = (
        f"You are an AI email assistant.\n"
        f"Reply style: {req.reply_style}\n\n"
        f"Conversation history:\n{history_text.strip()}\n\n"
        f"The latest message from {last_sender} ({last_email}) is:\n"
        f"\"{last_msg}\"\n\n"
        f"Generate a clear, well-structured, and {req.reply_style.lower()} reply based on the entire conversation."
    )

    # Generate response
    response = ollama.chat(
        model="gemma:1b-instruct",
        messages=[{"role": "user", "content": prompt}]
    )

    return {"response": response["message"]["content"]}

In [6]:
import ollama

def generate_auto_reply(conversation_thread, reply_style="formal", model="gemma:2b"):
    """
    Generates an AI email reply based on a conversation thread.

    Parameters:
        conversation_thread (list of dict): List of messages with keys: sender_name, sender_email, message.
        reply_style (str): Style of the reply e.g., "formal", "friendly", etc.
        model (str): Ollama model to use.

    Returns:
        str: AI-generated reply message.
    """
    if len(conversation_thread) < 1:
        return "Conversation thread is empty."

    *history_messages, latest_message = conversation_thread

    # Build conversation history
    history_text = ""
    for msg in history_messages:
        history_text += f"{msg['sender_name']} ({msg['sender_email']}): {msg['message']}\n"

    # Last message to reply to
    last_sender = latest_message["sender_name"]
    last_email = latest_message["sender_email"]
    last_msg = latest_message["message"]

    # Construct prompt
    prompt = (
        f"You are an AI email assistant.\n"
        f"Reply style: {reply_style}\n\n"
        f"Conversation history:\n{history_text.strip()}\n\n"
        f"The latest message from {last_sender} ({last_email}) is:\n"
        f"\"{last_msg}\"\n\n"
        f"Generate a clear, well-structured, and {reply_style.lower()} reply based on the entire conversation."
    )

    # Call Ollama
    response = ollama.chat(
        model=model,
        messages=[{"role": "user", "content": prompt}]
    )

    return response["message"]["content"]


In [7]:
if __name__ == "__main__":
    sample_conversation = [
        {
            "sender_name": "Team Lead",
            "sender_email": "lead@example.com",
            "message": "Hi John, please make sure the design document is ready by Thursday."
        },
        {
            "sender_name": "John",
            "sender_email": "john@example.com",
            "message": "Sure, I’ll finish the initial draft by tomorrow."
        },
        {
            "sender_name": "Team Lead",
            "sender_email": "lead@example.com",
            "message": "Great. Also, include the flow diagrams."
        }
    ]

    reply = generate_auto_reply(sample_conversation, reply_style="formal")
    print("AI-generated reply:\n")
    print(reply)


AI-generated reply:

Dear Team Lead,

I understand that the design document is a priority, and I will work diligently to complete it by Thursday. I have already started working on the initial draft and will prioritize the inclusion of the flow diagrams in the next iteration.

Please let me know if you have any further specific instructions or if you would like to review the draft document at any stage.

Thank you for your understanding and support.

Sincerely,
John
